In [20]:
import pandas as pd

In [ ]:
traj = pd.read_csv('./input/synthetic_data_100000.csv',usecols=['lon','lat','uid','time'])
traj.sort_values(by=['uid','time'],inplace=True)
traj['elat'] = traj.groupby('uid')['lat'].shift(-1)
traj['elon'] = traj.groupby('uid')['lon'].shift(-1)
traj = traj.dropna()

In [ ]:
traj = traj[traj['uid']==0]


In [12]:
import osmnx as ox
import warnings
warnings.filterwarnings('ignore')
G = ox.load_graphml('./shanghai_road.graphml')

In [15]:
hwy_speeds={"residential": 35, "secondary": 50,
                    'primary': 50, "tertiary": 60}
 # 设置路网
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)  # 将路网转换为GeoDataFrame
# 连接两个节点只保留一条边
gdf_edges = gdf_edges.reset_index().drop_duplicates(
    subset=['u', 'v'], keep='first')
gdf_edges['key'] = 0
gdf_edges = gdf_edges.set_index(['u', 'v', 'key'])
# 重新构建路网
# 图属性,这里使用了之前的路网数据的图属性
G = ox.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs=G.graph)
# 路网最大强连通子图
G = ox.utils_graph.get_largest_component(G,
                                            strongly=True  # 是否强连通
                                            )
# 计算每条边出行时长
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)


In [16]:
import numpy as np
def find_travel_path(G, lon1, lat1, lon2, lat2, shortest=1, weight='travel_time'):
    # 找到两点间的最短出行路径
    # 由给定的坐标获取最近节点
    orig = ox.distance.nearest_nodes(G, X=lon1, Y=lat1)
    dest = ox.distance.nearest_nodes(G, X=lon2, Y=lat2)
    if shortest == 1:
        # 找到最短路径
        travel_route = ox.shortest_path(
            G, orig, dest, weight=weight)
    if shortest > 1:
        # 前k最短路径中选择一个
        routes = ox.k_shortest_paths(
            G, orig, dest, k=shortest, weight=weight)
        routes = list(routes)
        travel_route = routes[np.random.choice(range(len(routes)))]
    # 获取路径上的行驶时间
    travel_time = ox.utils_graph.get_route_edge_attributes(
        G, travel_route, attribute='travel_time')
    length = ox.utils_graph.get_route_edge_attributes(
        G, travel_route, attribute='length')
    # 将路径和行驶时间组合成字典
    route = {
        'travel_route': travel_route,
        'travel_time': travel_time,
        'length': length,
        'has_path': len(travel_route) > 1,
        'slon': lon1,
        'slat': lat1,
        'elon': lon2,
        'elat': lat2}
    
find_travel_path(G,traj.iloc[0]['lon'],traj.iloc[0]['lat'],traj.iloc[0]['elon'],traj.iloc[0]['elat'])

{'direct': False, 'travel_route': [476555419, 1324294395, 476555427, 1324295149, 476561951, 476555428, 476562299, 618170115, 618170117, 618181478, 847910000, 847910448, 847973084, 847973097, 618426647, 1891836062, 9846916284], 'travel_time': [3.8, 5.0, 3.7, 7.7, 1.9, 20.3, 45.3, 6.7, 111.1, 51.8, 93.2, 36.8, 14.4, 22.1, 32.9, 40.6], 'length': [40.742, 53.294000000000004, 39.024, 81.023, 20.127, 282.47499999999997, 629.032, 93.505, 1851.0129999999997, 862.904, 1553.485, 613.4500000000002, 199.814, 307.20599999999996, 456.995, 676.94], 'has_path': True, 'slon': 121.49120500000001, 'slat': 31.217311, 'elon': 121.538826, 'elat': 31.197233}


In [18]:
import hashlib
def generate_unique_key(slon, slat, elon, elat):
    """
    生成一个基于坐标的唯一索引。
    参数:
    slon -- 起始点的经度
    slat -- 起始点的纬度
    elon -- 目的地的经度
    elat -- 目的地的纬度
    
    返回:
    str -- 一个MD5哈希值，用作唯一索引。
    """
    # 将坐标连接成一个字符串
    coordinates_str = f"{slon},{slat},{elon},{elat}"
    # 使用MD5哈希算法生成唯一的哈希值
    return hashlib.md5(coordinates_str.encode('utf-8')).hexdigest()
print(generate_unique_key(traj.iloc[0]['lon'],traj.iloc[0]['lat'],traj.iloc[0]['elon'],traj.iloc[0]['elat']))


f92e1cc6d887dca21a86faf6083277d3


f92e1cc6d887dca21a86faf6083277d3
